In [ ]:
!sudo apt-get update
!sudo apt-get install cmake libssl-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [ ]:
!git clone https://github.com/open-quantum-safe/liboqs-python.git

Cloning into 'liboqs-python'...
remote: Enumerating objects: 663, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 663 (delta 183), reused 167 (delta 148), pack-reused 433 (from 1)
Receiving objects: 100% (663/663), 139.71 KiB | 5.59 MiB/s, done.
Resolving deltas: 100% (340/340), done.


In [ ]:
%cd liboqs-python
!python3 setup.py build
!python3 setup.py install

/content/liboqs-python
running build
running build_py
creating build/lib/oqs
copying oqs/__init__.py -> build/lib/oqs
copying oqs/rand.py -> build/lib/oqs
copying oqs/oqs.py -> build/lib/oqs
running egg_info
creating liboqs_python.egg-info
writing liboqs_python.egg-info/PKG-INFO
writing dependency_links to liboqs_python.egg-info/dependency_links.txt
writing requirements to liboqs_python.egg-info/requires.txt
writing top-level names to liboqs_python.egg-info/top_level.txt
writing manifest file 'liboqs_python.egg-info/SOURCES.txt'
reading manifest file 'liboqs_python.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'liboqs_python.egg-info/SOURCES.txt'
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use 

In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.1 MB/s eta 0:00:00


In [ ]:
import oqs
import string
import random
from Crypto.Cipher import AES

liboqs not found, installing it in /root/_oqs
5 4 3 2 1 
Done installing liboqs


In [ ]:
class KEMSigner:
  def __init__(self):
    self.sigalg = "Dilithium2"
    self.signer = oqs.Signature(self.sigalg)
    self.sig_public_key = None
    self.kemalg = "Kyber512"
    self.kem = oqs.KeyEncapsulation(self.kemalg)
    self.kem_public_key = None
    self.kem_shared_secret = None
    self.ttl = None

  def genSigKeypair(self):
    self.sig_public_key = self.signer.generate_keypair()

  def genKemKeypair(self):
    self.kem_public_key = self.kem.generate_keypair()

  def signMessage(self,message):
      return self.signer.sign(message)

  def verifyMessage(self,message,signature,public_key):
    return self.signer.verify(message,signature,public_key)

  def encap(self, public_key):
    ciphertext, self.kem_shared_secret = self.kem.encap_secret(public_key)
    return ciphertext

  def decap(self, ciphertext):
    self.kem_shared_secret = self.kem.decap_secret(ciphertext)
    return self.kem_shared_secret

In [ ]:
def AESEncrypt(key, message):
  cipher = AES.new(key, AES.MODE_EAX)
  nonce = cipher.nonce
  ciphertext, tag = cipher.encrypt_and_digest(message)

  return ciphertext, tag, nonce

def AESDecrypt(key, ciphertext, nonce):
  cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
  plaintext = cipher.decrypt(ciphertext)

  return plaintext

In [ ]:
def main():
  # Server generates random challenge
  N = 7
  message = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
  print(message)
  message = message.encode()
  print(message)

  # Initialize client/server + keypairs
  client = KEMSigner()
  client.genSigKeypair()
  client.genKemKeypair()
  server = KEMSigner()
  server.genSigKeypair()
  server.genKemKeypair()

  ## CHALLENGE
  # Server sends KEM ciphertext
  kem_cipher = server.encap(client.kem_public_key)
  # Client now gets shared secret
  client.decap(kem_cipher)

  print(f"Verify shared secret is the same: {server.kem_shared_secret == client.kem_shared_secret}")

  # Server sends encrypted challenge
  ciphertext, tag, nonce = AESEncrypt(server.kem_shared_secret, message)

  # Client decrypts
  challenge = AESDecrypt(client.kem_shared_secret, ciphertext, nonce)
  # Client signs decrypted challenge and sends its back
  cli_signature = client.signMessage(challenge)
  # Server verifies client signature
  auth_cli = server.verifyMessage(message,cli_signature,client.sig_public_key)

  print(f"Verify user: {auth_cli}")

main()

GX9HN0V
b'GX9HN0V'
Verify shared secret is the same: True
Verify user: True
